In [1]:
from ipywidgets import Dropdown
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models


try:
    dropdown = Dropdown(
        options=list_jumpstart_models("search_keywords includes Text Generation"),
        value="meta-textgeneration-llama-codellama-7b",
        description="Select a JumpStart text generation model:",
        style={"description_width": "initial"},
        layout={"width": "max-content"},
    )
    display(dropdown)
except:
    dropdown = None
    pass

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Dropdown(description='Select a JumpStart text generation model:', index=111, layout=Layout(width='max-content'…

In [4]:
if dropdown:
    model_id = dropdown.value
else:
    model_id = "meta-textgeneration-llama-codellama-7b"
model_version = "*"

In [5]:
model_id

'meta-textgeneration-llama-codellama-13b-python'

In [6]:
from sagemaker.jumpstart.model import JumpStartModel


model = JumpStartModel(model_id=model_id, model_version=model_version, instance_type="ml.g4dn.12xlarge")

Using model 'meta-textgeneration-llama-codellama-13b-python' with wildcard version identifier '*'. You can pin to version '2.2.1' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


In [ ]:
predictor = model.deploy(
    accept_eula=True
)  

# Invoke Model

In [ ]:
example_payloads = model.retrieve_all_examples()

In [ ]:
import jmespath


for payload in example_payloads:
    response = predictor.predict(payload.body)
    generated_text = jmespath.search(payload.raw_payload["output_keys"]["generated_text"], response)
    print("Input:\n", payload.body[payload.prompt_key])
    print("Output:\n", generated_text.strip())
    print("\n===============\n")

# Finetune with LoRA

In [1]:
# ! pip install datasets

In [7]:
from ipywidgets import Dropdown
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models


try:
    dropdown = Dropdown(
        options=list_jumpstart_models("search_keywords includes Text Generation"),
        value="meta-textgeneration-llama-codellama-7b",
        description="Select a JumpStart text generation model:",
        style={"description_width": "initial"},
        layout={"width": "max-content"},
    )
    display(dropdown)
except:
    dropdown = None
    pass

Dropdown(description='Select a JumpStart text generation model:', index=111, layout=Layout(width='max-content'…

In [8]:
if dropdown:
    model_id = dropdown.value
else:
    model_id = "meta-textgeneration-llama-codellama-7b"
model_version = "*"

model_id

'meta-textgeneration-llama-codellama-13b-python'

In [2]:
from datasets import load_dataset


dolphin = load_dataset("cognitivecomputations/dolphin-coder", split="train")

# We split the dataset into two where test data is used to evaluate at the end.
train_and_test_dataset = dolphin.train_test_split(test_size=0.9, seed=0)

# Dumping the training data to a local file to be used for training.
train_and_test_dataset["train"].to_json("./dolphin-coder-dataset/train.jsonl")
train_and_test_dataset["test"].select(range(10)).to_json("./dolphin-coder-dataset/test.jsonl")

Generating train split: 0 examples [00:00, ? examples/s]

Creating json from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

31601

In [3]:
train_and_test_dataset["train"][0]

{'system_prompt': 'a chat',
 'question': 'Please explain the following problem in detail and provide code in Haskell:\nYou are given a 2D integer array `ranges` and two integers `left` and `right`. Each `ranges[i] = [starti, endi]` represents an **inclusive** interval between `starti` and `endi`.\n\nReturn `true` _if each integer in the inclusive range_ `[left, right]` _is covered by **at least one** interval in_ `ranges`. Return `false` _otherwise_.\n\nAn integer `x` is covered by an interval `ranges[i] = [starti, endi]` if `starti <= x <= endi`.\n\n**Example 1:**\n\n**Input:** ranges = \\[\\[1,2\\],\\[3,4\\],\\[5,6\\]\\], left = 2, right = 5\n**Output:** true\n**Explanation:** Every integer between 2 and 5 is covered:\n- 2 is covered by the first range.\n- 3 and 4 are covered by the second range.\n- 5 is covered by the third range.\n\n**Example 2:**\n\n**Input:** ranges = \\[\\[1,10\\],\\[10,20\\]\\], left = 21, right = 21\n**Output:** false\n**Explanation:** 21 is not covered by any

In [4]:
import json

template = {
    "prompt": """{system_prompt}

### Input:
{question}
""",
    "completion": " {response}",
}
with open("./dolphin-coder-dataset/template.json", "w") as f:
    json.dump(template, f)

In [5]:
from sagemaker.s3 import S3Uploader
import sagemaker
import random

output_bucket = "sandbox-dump"
local_data_file = "./dolphin-coder-dataset/train.jsonl"
train_data_location = f"s3://{output_bucket}/codellama-test/dolphin_coder_dataset"
S3Uploader.upload(local_data_file, train_data_location)
S3Uploader.upload("./dolphin-coder-dataset/template.json", train_data_location)
print(f"Training data: {train_data_location}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Training data: s3://sandbox-dump/codellama-test/dolphin_coder_dataset


In [9]:
from sagemaker import hyperparameters

my_hyperparameters = hyperparameters.retrieve_default(
    model_id=model_id, model_version=model_version
)

print(my_hyperparameters)

Using model 'meta-textgeneration-llama-codellama-13b-python' with wildcard version identifier '*'. You can pin to version '2.2.1' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


{'int8_quantization': 'False', 'enable_fsdp': 'True', 'epoch': '2', 'learning_rate': '0.0001', 'lora_r': '8', 'lora_alpha': '32', 'target_modules': 'q_proj,v_proj', 'lora_dropout': '0.05', 'instruction_tuned': 'True', 'chat_dataset': 'False', 'add_input_output_demarcation_key': 'True', 'per_device_train_batch_size': '2', 'per_device_eval_batch_size': '1', 'max_train_samples': '-1', 'max_val_samples': '-1', 'seed': '10', 'max_input_length': '2048', 'validation_split_ratio': '0.2', 'train_data_split_seed': '0', 'preprocessing_num_workers': 'None'}


In [10]:
my_hyperparameters["epoch"] = "1"
print(my_hyperparameters)

hyperparameters.validate(
    model_id=model_id, model_version=model_version, hyperparameters=my_hyperparameters
)

{'int8_quantization': 'False', 'enable_fsdp': 'True', 'epoch': '1', 'learning_rate': '0.0001', 'lora_r': '8', 'lora_alpha': '32', 'target_modules': 'q_proj,v_proj', 'lora_dropout': '0.05', 'instruction_tuned': 'True', 'chat_dataset': 'False', 'add_input_output_demarcation_key': 'True', 'per_device_train_batch_size': '2', 'per_device_eval_batch_size': '1', 'max_train_samples': '-1', 'max_val_samples': '-1', 'seed': '10', 'max_input_length': '2048', 'validation_split_ratio': '0.2', 'train_data_split_seed': '0', 'preprocessing_num_workers': 'None'}


In [ ]:
my_hyperparameters["pretrained-model"] = "1"
my_hyperparameters

In [11]:
from sagemaker.jumpstart.estimator import JumpStartEstimator


estimator = JumpStartEstimator(
    model_id=model_id,
    model_version=model_version,
    hyperparameters=my_hyperparameters,
    instance_type="ml.g4dn.12xlarge",
    source_dir = "s3://sandbox-dump/llama-codellama-13b-python-2024-03-24-12-34-55-192/source/sourcedir.tar.gz",
    # source_dir = "./src",
    entry_point = "transfer_learning.py",
    output_path = "s3://sandbox-dump/codellama-test/output",
    checkpoint_s3_uri = "s3://sandbox-dump/codellama-test/checkpoints",
    environment={
        "accept_eula": "true"
    },  # please change `accept_eula` to be `true` to accept EULA.
)

estimator.fit({"training": train_data_location})

INFO:sagemaker:Creating training-job with name: llama-codellama-13b-python-2024-03-24-12-34-55-192


2024-03-24 12:36:35 Starting - Starting the training job...
2024-03-24 12:36:52 Starting - Preparing the instances for training......
2024-03-24 12:38:02 Downloading - Downloading input data......
2024-03-24 12:39:02 Downloading - Downloading the training image.....................
2024-03-24 12:42:28 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-03-24 12:42:56,492 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-03-24 12:42:56,556 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-03-24 12:42:56,565 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-03-24 12:42:56,566 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-03-24 12:43:05,741 sagemaker-training-toolkit INFO     Installing de

UnexpectedStatusException: Error for Training job llama-codellama-13b-python-2024-03-24-12-34-55-192: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "raise EnvironmentError(
 OSError
 Can't load tokenizer for 'None'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'None' is the correct path to a directory containing all relevant files for a LlamaTokenizerFast tokenizer.
 INFO:root:Loading the tokenizer.
 Traceback (most recent call last)
 File "/opt/ml/code/llama_finetuning.py", line 337, in <module>
 fire.Fire(main)
 File "/opt/conda/lib/python3.10/site-packages/fire/core.py", line 141, in Fire
 fire.Fire(main)fire.Fire(main)
 
 component_trace = _Fire(component, args, parsed_flag_args, context, name)
 File "/opt/conda/lib/python3.10/site-packages/fire/core.py", line 475, in _Fire
 component_trace = _Fire(component, args, parsed_flag_args, context, n

# Deploy Fine-Tuned Model

In [ ]:
finetuned_predictor = estimator.deploy()

In [ ]:
import pandas as pd
from IPython.display import display, HTML

test_dataset = load_dataset("json", data_files="test.jsonl")["train"]
prompt_inference = template["prompt"]
inputs, ground_truth_responses, responses_before_finetuning, responses_after_finetuning = (
    [],
    [],
    [],
    [],
)


def predict_and_print(datapoint):
    # For instruction fine-tuning, we insert a special key between input and output
    input_output_demarkation_key = "\n\n### Response:\n"

    payload = {
        "inputs": prompt_inference.format(
            system_prompt=datapoint["system_prompt"], question=datapoint["question"]
        )
        + input_output_demarkation_key,
        "parameters": {"max_new_tokens": 100},
    }
    inputs.append(payload["inputs"])
    ground_truth_responses.append(datapoint["response"])
    pretrained_response = predictor.predict(payload)
    responses_before_finetuning.append(pretrained_response[0]["generated_text"])
    finetuned_response = finetuned_predictor.predict(payload)
    responses_after_finetuning.append(finetuned_response[0]["generated_text"])


try:
    for i, datapoint in enumerate(test_dataset.select(range(5))):
        predict_and_print(datapoint)

    df = pd.DataFrame(
        {
            "Inputs": inputs,
            "Ground Truth": ground_truth_responses,
            "Response from non-finetuned model": responses_before_finetuning,
            "Response from fine-tuned model": responses_after_finetuning,
        }
    )
    display(HTML(df.to_html()))
except Exception as e:
    print(e)

In [ ]:
predictor.delete_predictor()
finetuned_predictor.delete_predictor()